In [47]:
import matplotlib.pyplot as plt
import pandas as pd

import numpy as np
import datetime as dt

import wbdata
# we hates modules that don't apppear on anaconda!
# pip install wbdata
import datetime
import csv
import statsmodels.api as sm
from statsmodels.api import add_constant

# from pandas.stats.plm import PanelOLS
# deprecated

from linearmodels import PanelOLS
# pip install linearmodels

In [43]:
import os
BES_data_folder   = "../BES_analysis_data/"
if not os.path.exists( BES_data_folder ):
    os.makedirs( BES_data_folder )
    
Legatum_folder = BES_data_folder + os.sep + "Legatum" + os.sep
encoding = "ISO-8859-1"

In [44]:
## pull in and clean fdi stock data ##

# fdi = pd.read_excel("us_fdiflowsstock_29777289490548.xls", header = 4)
# save as csv! xls is a pain
# header=4 -> header=5
fdi_raw = pd.read_csv(Legatum_folder+"us_fdiflowsstock_29777289490548.csv", header=5, encoding=encoding)
fdi = fdi_raw.iloc[1:]
fdi = fdi_raw.transpose()

fdi.columns = fdi.iloc[0]
fdi = fdi.iloc[1:]
fdi = fdi.stack()
fdi = fdi.reset_index()

fdi.columns = ['Year', 'country', 'FDI stock']
fdi['Year'] = fdi['Year'].astype('int', errors='raise')
fdi['date'] = pd.to_datetime(fdi['Year'], errors='ignore', format = "%Y")
fdi['FDI stock'] = pd.to_numeric(fdi['FDI stock'], errors='coerce')
fdi['country'] = fdi['country'].str.strip()
##### sort -> sort_values
fdi = fdi.sort_values(["country", "date"])

## pull year from annual data to ease matching to rest ## 
fdi['year'] = pd.DatetimeIndex(fdi['date']).year
fdi = fdi.set_index(["country", "year"])
del fdi['Year']
del fdi['date']

## pull remaining data from the world bank ##

indicators = {"BN.KLT.DINV.CD" : "FDI", "NY.GDP.MKTP.CD" : "GDP per capita", "SH.XPD.CHEX.PC.CD": "Health expenditure", "FS.AST.PRVT.GD.ZS" : "Domestic credit", "TX.VAL.FUEL.ZS.UN" : "Fuel exports", "TX.VAL.MMTL.ZS.UN" : "Ore and metal", "se.prm.cmpt.zs": "Primary school completion"}
df = wbdata.get_dataframe(indicators, country="all", data_date=(datetime.datetime(2010, 1, 1), datetime.datetime(2018, 1, 1)),  convert_date=True)

## pull year from annual data to ease matching to fdi ## 
df = df.reset_index()
df['year'] = pd.DatetimeIndex(df['date']).year
df = df.set_index(["country", "year"])
del df['date']

df = df.join(fdi)
df = df.dropna()

df['GDP per capita log'] = df['GDP per capita'].apply(np.log)
df['FDI log'] = df['FDI stock'].apply(np.log)

## this caps the number of obs per country at 4 - gives you 379 rows (their n = 384)##

df = df.reset_index()
df_2 = df.groupby("country").head(4)
df = df_2
df = df.set_index(["country", "year"])

In [45]:
df
# passes eyeball comparison with xtophercooks notebook

Domestic credit           FDI  Fuel exports  \
country             year                                                
Albania             2015        37.151375 -9.100003e+08      8.806638   
                    2014        39.261063 -1.073931e+09      1.567523   
                    2013        39.787091 -1.225802e+09     31.003052   
                    2012        40.803822 -8.350794e+08     26.590781   
Algeria             2015        21.725470  4.989344e+08     94.325860   
                    2014        18.351978 -1.521892e+09     95.764232   
                    2013        16.497891 -1.963635e+09     96.722000   
                    2012        14.025307 -1.541845e+09     97.135491   
Angola              2011        20.179427  5.116413e+09     96.556768   
                    2010        20.202350  4.567641e+09     96.534547   
Antigua and Barbuda 2015        51.219357 -9.564097e+07      0.000000   
                    2014        60.348025 -4.001160e+07      0.000000   
Argentina           2015        14.414233 -1.088376e+10      2.611328   
                    2014        13.823769 -3.144794e+09      4.751054   
                    2013        15.729092 -8.931689e+09      4.758954   
                    2012        15.212815 -1.426908e+10      6.374852   
Armenia             2015        45.630783 -1.615249e+08      6.419701   
Austria             2015        85.835374  6.361436e+09      1.967985   
                    2014        87.577525 -2.586685e+09      2.386196   
                    2013        92.557752  1.057847e+10      2.658385   
                    2012        94.223799  1.323003e+10      3.624504   
Azerbaijan          2015        38.520786 -8.382520e+08     87.023136   
                    2014        30.638025 -2.443393e+09     92.638413   
                    2013        25.211603 -1.141550e+09     92.985039   
                    2012        19.811328 -8.124070e+08     93.419357   
Bahrain             2015        73.717543 -3.126064e+09     58.144737   
                    2014        63.878768  1.913032e+09     66.834655   
                    2013        69.630491  3.196064e+09     59.005034   
                    2012        69.143572  1.028723e+09     64.148803   
Bangladesh          2015        44.406975 -2.771251e+09      0.561580   
...                                   ...           ...           ...   
Togo                2013        37.676552 -2.042211e+08      0.995085   
                    2012        30.146509  2.988119e+08      2.017440   
Tonga               2013        30.679238 -4.475877e+06      0.164329   
                    2012        30.513391 -2.869190e+05      0.000628   
Trinidad and Tobago 2010        30.845461 -5.494000e+08     66.122752   
Tunisia             2015        79.338136 -9.705219e+08      7.211910   
                    2014        78.116264 -1.024754e+09     13.189847   
                    2013        76.956587 -1.058623e+09     15.216031   
                    2010        69.111924 -1.334498e+09     14.177306   
Turkey              2015        66.827360 -1.290600e+10      3.083329   
                    2014        63.791062 -6.069000e+09      3.761938   
                    2012        52.228001 -9.638000e+09      5.285242   
                    2011        49.424062 -1.381200e+10      4.699200   
Uganda              2014        14.872813 -1.031546e+09      2.114100   
                    2013        13.913392 -1.143207e+09      1.073146   
                    2012        13.983883 -1.159051e+09      6.673171   
                    2011        15.803888 -9.061404e+08      5.773000   
Ukraine             2014        75.228846 -2.990000e+08      3.389940   
                    2013        73.463387 -4.079000e+09      4.200171   
                    2012        69.579246 -7.195000e+09      5.049094   
                    2011        71.075042 -7.015000e+09      8.112370   
United Kingdom      2015       132.787995 -1.145039e+11      7.654935   
Uruguay    

In [54]:
Y = df["GDP per capita log"]
X = df[["FDI log", "Health expenditure", "Domestic credit", "Fuel exports", "Ore and metal", "Primary school completion"]]
X = add_constant(X)

# regression=PanelOLS(y=Y, x=X, time_effects=True)
# regression

# linearmodel PanelOLS is *not completely identical* to pandas
# their explanation of the differences
# https://bashtage.github.io/linearmodels/doc/panel/pandas.html

PanelOLS(Y, X, time_effects=True).fit()

Dep. Variable:,GDP per capita log,R-squared:,0.8001
Estimator:,PanelOLS,R-squared (Between):,0.8251
No. Observations:,379,R-squared (Within):,-3.0971
Date:,"Tue, Sep 25 2018",R-squared (Overall):,0.7982
Time:,02:30:31,Log-likelihood,-502.21
Cov. Estimator:,Unadjusted,,
,,F-statistic:,244.78
Entities:,112,P-value,0.0000
Avg Obs:,3.3839,Distribution:,"F(6,367)"
Min Obs:,1.0000,,
Max Obs:,4.0000,F-statistic (robust):,244.78
